## Neural networks project

#### This is a datascience training project in which I had to create a neural network to predict the kind of trees will grow in an area based on cartographic data

### 1st I loaded the necessary libraries:

In [16]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import  Dense, Dropout


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

### Then I loaded and inspected the data, as usual:

In [17]:
data = pd.read_csv("cover_data.csv")

In [18]:
data.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,class
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


### To avoid writing all the features one-by-one, this command gives me a list of columns, I'll just need to remove the last one that gives the tree-cover type, which is what we are training the algorithm to predict:

In [19]:
data.columns

Index(['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40

### This command gets me a list of all the cartographic data into a dataframe, this is going to be our input in the neural network:

In [20]:
features = data[['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4',
       'Soil_Type1', 'Soil_Type2', 'Soil_Type3', 'Soil_Type4', 'Soil_Type5',
       'Soil_Type6', 'Soil_Type7', 'Soil_Type8', 'Soil_Type9', 'Soil_Type10',
       'Soil_Type11', 'Soil_Type12', 'Soil_Type13', 'Soil_Type14',
       'Soil_Type15', 'Soil_Type16', 'Soil_Type17', 'Soil_Type18',
       'Soil_Type19', 'Soil_Type20', 'Soil_Type21', 'Soil_Type22',
       'Soil_Type23', 'Soil_Type24', 'Soil_Type25', 'Soil_Type26',
       'Soil_Type27', 'Soil_Type28', 'Soil_Type29', 'Soil_Type30',
       'Soil_Type31', 'Soil_Type32', 'Soil_Type33', 'Soil_Type34',
       'Soil_Type35', 'Soil_Type36', 'Soil_Type37', 'Soil_Type38',
       'Soil_Type39', 'Soil_Type40']]

### I also need to create a dataframe with the labels, which is what we want to predict with the algorythm:

In [21]:
labels = data["class"]

### Looking at the shape of the labels, it seems that type 1 and type 2 treecovers are the most abundant:

In [22]:
labels.value_counts()

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: class, dtype: int64

### In most machine learning models, including neural networks, we usually start by dividing the data into a training set and a testing set. 

### Like the name indicates, the training set is used to train the neural network, and the test set is used to evaluate it's accuracy:

In [23]:
x_train, x_test, y_train, y_test = train_test_split(features, labels)

### Before we start, we need to make sure that the data is normalized, we don't want any input to have disproportionate weight. We'll normalize it in a way that it's average becomes 0 and std deviation becomes 1:

In [24]:
ct = StandardScaler()
x_train = ct.fit_transform(x_train)
x_test = ct.transform(x_test)

### Now we are ready to create the neural network, we start by chosing which kind of model:

In [25]:
model = Sequential()

### We create the imput layer, which corresponds to the number of features:

In [26]:
model.add(tf.keras.layers.InputLayer(input_shape=(54,)))

### Then we build a brain, this is the customizable step, and where the name deep learning comes from, we can decide how deep the brain is, and how many neurons it contains, balancing the accuracy of the model and the time it takes to compute.

### After some configuration attempts, this is the brain architecture I came up with: 

In [27]:
model.add(Dense(356, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(8, activation='softmax'))

### We create a script to train the neural network, with the parameters we are optimizing:

In [28]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

### And we input our training data, with 100 epochs the accuracy of the model is 93%. You can reduced the number of epochs if you are in a hurry, althought the accuracy won't be as good:

In [29]:
model.fit(x_train, y_train, epochs = 100, batch_size = 5000)

Epoch 1/100
88/88 [==============================] - 5s 47ms/step - loss: 0.8234 - sparse_categorical_accuracy: 0.6852
Epoch 2/100
88/88 [==============================] - 4s 46ms/step - loss: 0.5769 - sparse_categorical_accuracy: 0.7560
Epoch 3/100
88/88 [==============================] - 4s 45ms/step - loss: 0.5223 - sparse_categorical_accuracy: 0.7779
Epoch 4/100
88/88 [==============================] - 4s 40ms/step - loss: 0.4865 - sparse_categorical_accuracy: 0.7935
Epoch 5/100
88/88 [==============================] - 4s 43ms/step - loss: 0.4570 - sparse_categorical_accuracy: 0.8076
Epoch 6/100
88/88 [==============================] - 4s 44ms/step - loss: 0.4335 - sparse_categorical_accuracy: 0.8194
Epoch 7/100
88/88 [==============================] - 4s 44ms/step - loss: 0.4157 - sparse_categorical_accuracy: 0.8273
Epoch 8/100
88/88 [==============================] - 4s 43ms/step - loss: 0.3983 - sparse_categorical_accuracy: 0.8353
Epoch 9/100
88/88 [=============================

88/88 [==============================] - 3s 39ms/step - loss: 0.1947 - sparse_categorical_accuracy: 0.9230
Epoch 70/100
88/88 [==============================] - 4s 40ms/step - loss: 0.1992 - sparse_categorical_accuracy: 0.9205
Epoch 71/100
88/88 [==============================] - 4s 42ms/step - loss: 0.1873 - sparse_categorical_accuracy: 0.9262
Epoch 72/100
88/88 [==============================] - 4s 40ms/step - loss: 0.1874 - sparse_categorical_accuracy: 0.9260
Epoch 73/100
88/88 [==============================] - 3s 38ms/step - loss: 0.1845 - sparse_categorical_accuracy: 0.9280
Epoch 74/100
88/88 [==============================] - 3s 39ms/step - loss: 0.1858 - sparse_categorical_accuracy: 0.9271
Epoch 75/100
88/88 [==============================] - 4s 49ms/step - loss: 0.1876 - sparse_categorical_accuracy: 0.9262
Epoch 76/100
88/88 [==============================] - 4s 46ms/step - loss: 0.1878 - sparse_categorical_accuracy: 0.9258
Epoch 77/100
88/88 [==============================] -

### And we test the model on our validation data, getting a final report about the performance of the model:

In [30]:
y_estimate = model.predict(x_test)
y_estimate = np.argmax(y_estimate, axis = 1)
print(classification_report(y_test, y_estimate))

4540/4540 [==============================] - 7s 1ms/step
              precision    recall  f1-score   support

           1       0.96      0.87      0.91     52978
           2       0.90      0.96      0.93     70810
           3       0.87      0.95      0.91      8975
           4       0.85      0.81      0.83       690
           5       0.80      0.79      0.80      2308
           6       0.91      0.73      0.81      4386
           7       0.93      0.94      0.94      5106

    accuracy                           0.92    145253
   macro avg       0.89      0.87      0.88    145253
weighted avg       0.92      0.92      0.92    145253



### Final comment: neural networks have the advantage of being very efficient at classifing data, but they have the problem of being "blackboxes", in which it is almost impossible to get insight on what is the weight of each input to the final prediction.

### Usually, when trying to understand the why is more important than the final classification, using other machine learning methods is more appropriate.